<a href="https://colab.research.google.com/github/okana2ki/ML/blob/main/20231220_SwallowLLM_rev1_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 資料の出典など
このノートブックは、[このノートブック](https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/20231220_SwallowLLM.ipynb)に岡が★加筆★したものです。

2023/12/19のSwallowに対応しています。その後、2024/7に[Llama 3 Swallow](https://swallow-llm.github.io/llama3-swallow.ja.html)が発表されていますが、これには対応していません。

# Demo for Swallow LLM by Tokyo Tech

公式サイト
https://tokyotech-llm.github.io/

モデル
https://huggingface.co/tokyotech-llm

★AICU media 「[東工大と産総研、英語の言語理解や対話で高い能力を持つ大規模言語モデル「Swallow」を公開 #SwallowLLM](https://note.com/aicu/n/n3eb8c1f2df02)」

★AICU media 「[東工大LLM「Swallow」を使ってGoogle Colabで遊んでみよう #SwallowLLM](https://note.com/aicu/n/nd0337d4952f3)」の解説コードです

参考：比較的初心者向けのGoogle Colabでの「Japanese Stable LM Gamma 7B」を動かす記事

■[Stability AI Japanが公開した30億パラメーターの日本語向けLLMを動かしてみた - 生成AIストリーム - 窓の杜](https://forest.watch.impress.co.jp/docs/serial/aistream/1544320.html)

Coded by Akihiko SHIRAI (kaitas[@o_ob](https://twitter.com/o_ob)) PoC開発や技術発信のお仕事歓迎です

# Step.1

[Google Colab Pro](https://colab.research.google.com/signup/pricing?hl=ja) を使ってGPUが利用できるインスタンスを作ります。具体的には「A100 GPU」もしくは「V100 GPU」以上を選びましょう。「A100」を選んでしばらく待てば、運が良ければ割り当てられます（利用できない時は「V100」になります）。

★無料版のColabでも使用量の制限がありますが、T4 GPUが利用できます。画面上部の「ランタイム」をクリックし、ドロップダウンメニューから「ランタイムのタイプを変更」をクリック、そこで「T4 GPU」を選択します。無料版の場合は、Step.2で「7b-instruct」を選んでください（[参考資料](https://ex-gram.com/llm-swallow/)）。★

In [ ]:
# パッケージのインストール、いろいろあるけど Colab環境ならこれだけで動くはず
!pip install sentencepiece accelerate

In [ ]:
# @title Step.2 Tokenizer & Model Loading
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# @markdown [https://huggingface.co/tokyotech-llm](https://huggingface.co/tokyotech-llm) から利用したいモデルを選択してください。最初は 7b-instruct から始めるのがおすすめです。13bは ColabPro では動いています。70bはColabProでもダウンロードが難しいです。
tokenizer_model_name = "tokyotech-llm/Swallow-7b-instruct-hf" # @param ['tokyotech-llm/Swallow-7b-hf','tokyotech-llm/Swallow-7b-instruct-hf','tokyotech-llm/Swallow-13b-hf','tokyotech-llm/Swallow-13b-instruct-hf','tokyotech-llm/Swallow-70b-hf','tokyotech-llm/Swallow-70b-instruct-hf']
model_name = "tokyotech-llm/Swallow-7b-instruct-hf" # @param ['tokyotech-llm/Swallow-7b-hf','tokyotech-llm/Swallow-7b-instruct-hf','tokyotech-llm/Swallow-13b-hf','tokyotech-llm/Swallow-13b-instruct-hf','tokyotech-llm/Swallow-70b-hf','tokyotech-llm/Swallow-70b-instruct-hf']

# Load
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True ,device_map="auto")

# @title
PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"

    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}

def create_prompt(instruction, input=None):
    """
    Generates a prompt based on the given instruction and an optional input.
    If input is provided, it uses the 'prompt_input' template from PROMPT_DICT.
    If no input is provided, it uses the 'prompt_no_input' template.

    Args:
        instruction (str): The instruction describing the task.
        input (str, optional): Additional input providing context for the task. Default is None.

    Returns:
        str: The generated prompt.
    """
    if input:
        # Use the 'prompt_input' template when additional input is provided
        return PROMPT_DICT["prompt_input"].format(instruction=instruction, input=input)
    else:
        # Use the 'prompt_no_input' template when no additional input is provided
        return PROMPT_DICT["prompt_no_input"].format(instruction=instruction)


In [7]:
# @title Step.3 Settings & Prompts
instruction_example = "以下のトピックに関する詳細な情報を提供してください。" # @param {type: "string"}
input_example = "七夕の起源について教えて" # @param {type: "string"}
Do_sample=True #@param {type:"boolean"}

if Do_sample:
  temperature = 0.1 #@param {type:"slider", min:0, max:2, step:0.1}
  top_p = 0.95 #@param {type:"slider", min:0, max:1, step:0.01}

max_new_tokens=128 #@param {type:"slider", min:128, max:1024, step:64}

# Example usage
# instruction_example = "以下のトピックに関する詳細な情報を提供してください。"
# input_example = "東京工業大学の主なキャンパスについて教えてください"

# 7b-instruct-hfモデルを使う場合、次のプロンプトを使用：
prompt = create_prompt(instruction_example, input_example)

# ★7b-hfモデルを使う場合、次のようにプロンプトを入力（これは一例なので適当に書き換える）：
# prompt = '昔々あるところに'

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    top_p=top_p,
    do_sample=Do_sample,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)


以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
以下のトピックに関する詳細な情報を提供してください。

### 入力:
七夕の起源について教えて

### 応答:七夕は、中国の伝説に由来する日本の伝統的な祝日です。この日は、織姫と彦星が年に一度再会するという伝説に基づいています。この日は、7月7日に祝われ、日本では「七夕」と呼ばれています。


## ★Step.3のパラメータの説明（[参考資料](https://techblog.a-tm.co.jp/entry/2023/04/24/181232)）★

大規模言語モデルは、**確率分布に従ってランダムに次のトークンを生成する**が、temperatureは、確率分布の性質（１点集中型か／差がつきにくくするか）を決めるパラメータで、top_pは、確率分布からの選択の仕方（上位どこまでの範囲を候補に残すか）を決めるパラメータ。
* temperature: 温度が低いと選択確率に差が付きやすく、一番もっともらしい候補を常に選ぶ傾向。温度が高いと選択確率が一様に近づく（いろいろな応答をする）傾向。
* top_p: 累積確率がpまでの上位トークンだけが選択対象。top_p=1であればすべてのトークンが選択対象。
* max_new_tokens: モデルからの出力の最大トークン数。

# 演習問題１
temperatureやtop_pのパラメータを変えて、いろいろな入力を試し、パラメータによる出力の変化を観察し、結果を報告しなさい。

input:七夕の起源について教えて\
温度：0.99\
top_p: 0.95\
出力：七夕は、中国の民間伝承にルーツを持つ風習で、夏の半ば頃に行われるものです。中国の民話には、織姫と呼ばれる天女と彦星と呼ばれる青年の物語が出てきます。織姫は天帝の娘で、彦星は牛飼いであり、天帝は二人を結婚させました。結婚してからも、二人は天の仕事を続けていたが、それが原因で二人の間には疎遠感が募っていく。ある夜、牛が暴れ、その鳴き声

温度：1.9\
出力：昔、七日ごとに神を祀り儀式をする祭りが月、7つもあり、おかしが7軒に行き渡るほどの長い棒が一列に立てます\
出力：いいアイデアだ七月節八撈の始まりを起算させ、「乞巧伝系譜七月節七大月」と改編し、現在「七大月七」で呼ばれている。

温度：0.1\
出力：七夕は、中国の伝説に由来する日本の伝統的な祝日です。この日は、織姫と彦星が年に一度再会するという伝説に基づいています。この日は、7月7日に祝われ、日本では「七夕」と呼ばれています。

# 演習問題２
## ２－１


## ２－２
Step.2で7b-hfモデルを選んで、同じプロンプト入力かつ同じパラメータに対する出力を、7b-instruct-hfモデルの出力（演習問題１での出力）と比べて、結果を報告しなさい。

## ２－３
Step.3のプログラム中の下記の部分の、上の方のpromptへの代入文をコメントアウトして、下の方のpromptへの代入文の先頭の`# `を外して、自由入力形式のプロンプト入力で実行することにより、7b-instruct-hfモデルと7b-hfモデルの間での出力の違いを観察し、結果を報告しなさい。なお、コメント行にするかしないか（先頭に'# 'を付けるかどうか）は、該当行において'CTRL'と'/'を同時に押すことで切り替えることができる）
```
# 7b-instruct-hfモデルを使う場合、次のプロンプトを使用：
prompt = create_prompt(instruction_example, input_example)

# ★7b-hfモデルを使う場合、次のようにプロンプトを入力（これは一例なので適当に書き換える）：
# prompt = '昔々あるところに'
```



（ここに回答）